# Setup

Import a few common modules

In [ ]:
#CODE for points = 1
#import sklearn, numpy, os

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
import numpy as np
import sklearn
import os

%load_ext tensorboard



# to make this notebook's output stable across runs
np.random.seed(42)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Vanishing/Exploding Gradients Problem

In [ ]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
z = np.linspace(-5, 5, 200)

## Xavier and He Initialization

In [ ]:
#CODE for points = 1
# set activation to relu and kernel initializer to he_normal

keras.layers.Dense(10, activation= "relu", kernel_initializer= "he_normal")

### Leaky ReLU

In [ ]:
#CODE for points = 1

# remember α is the hyperparameter that defines how much the function “leaks”
# set the value of alpha, use the value typically set

def leaky_relu(z, alpha= .01):
    return np.maximum(alpha*z, z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [ ]:
#CODE for points = 1
#load MNIST dataset from keras

import keras


(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
#CODE for points = 1

# initialize kernel_initializer to "he_normal" and activation function to softmax

tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer= "he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer= "he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation= "softmax")
])

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.7955 - accuracy: 0.7361 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8200
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5553 - accuracy: 0.8156 - val_loss: 0.5350 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5339 - accuracy: 0.8223 - val_loss: 0.5156 - val_accuracy:

Now look at what happens if we try to use the ReLU activation function instead:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))

model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 16s 9ms/step - loss: 1.2697 - accuracy: 0.8755 - val_loss: 0.8163 - val_accuracy: 0.8900
Epoch 2/5
1719/1719 [==============================] - 14s 8ms/step - loss: 0.8188 - accuracy: 0.8848 - val_loss: 0.7755 - val_accuracy: 0.9020
Epoch 3/5
1719/1719 [==============================] - 15s 9ms/step - loss: 0.7986 - accuracy: 0.8913 - val_loss: 0.7645 - val_accuracy: 0.9048
Epoch 4/5
1719/1719 [==============================] - 15s 9ms/step - loss: 0.7897 - accuracy: 0.8952 - val_loss: 0.7560 - val_accuracy: 0.9068
Epoch 5/5
1719/1719 [==============================] - 16s 9ms/step - loss: 0.7848 - accuracy: 0.8958 - val_loss: 0.7567 - val_accuracy: 0.9060


Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_105 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_106 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [ ]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [ ]:
#bn1.updates #deprecated

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 7ms/step - loss: 0.8619 - accuracy: 0.7335 - val_loss: 0.4687 - val_accuracy: 0.8716
Epoch 2/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4346 - accuracy: 0.8734 - val_loss: 0.3494 - val_accuracy: 0.9038
Epoch 3/5
1719/1719 [==============================] - 9s 6ms/step - loss: 0.3559 - accuracy: 0.8965 - val_loss: 0.3001 - val_accuracy: 0.9174
Epoch 4/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3115 - accuracy: 0.9101 - val_loss: 0.2695 - val_accuracy: 0.9242
Epoch 5/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.2815 - accuracy: 0.9175 - val_loss: 0.2444 - val_accuracy: 0.9306


Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4748 - accuracy: 0.8401 - val_loss: 0.4176 - val_accuracy: 0.8572
Epoch 2/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4601 - accuracy: 0.8412 - val_loss: 0.4052 - val_accuracy: 0.8594
Epoch 3/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4494 - accuracy: 0.8454 - val_loss: 0.3969 - val_accuracy: 0.8628
Epoch 4/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4351 - accuracy: 0.8503 - val_loss: 0.3896 - val_accuracy: 0.8652
Epoch 5/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.4236 - accuracy: 0.8524 - val_loss: 0.3832 - val_accuracy: 0.8688


# Faster Optimizers

## Momentum optimization

In [ ]:
#CODE for points = 1
# initialize lr and momentum to typical values

optimizer = optimizers.SGD(learning_rate= .01, momentum= .9)

## Nesterov Accelerated Gradient

In [ ]:
#CODE for points = 1
# initialize lr and momentum to typical values. Set nesterov so that it is used

optimizer = optimizers.SGD(learning_rate=.01, momentum= .9, nesterov=.1)

# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [ ]:
#CODE for points = 0.5

from tensorflow.keras import regularizers

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(l2=0.01))

In [ ]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/
# nadam optimizer

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=tf.keras.regularizers.l2(l2=0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer= "nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 16s 9ms/step - loss: 1.2773 - accuracy: 0.8766 - val_loss: 0.8154 - val_accuracy: 0.8884
Epoch 2/2
1719/1719 [==============================] - 14s 8ms/step - loss: 0.8188 - accuracy: 0.8850 - val_loss: 0.7758 - val_accuracy: 0.9030


In [ ]:
#CODE for points = .5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(l2=0.01))

In [ ]:
#CODE for points = 1

# activation function initialized as softmax
# nadam optimizer


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation= "softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))


Epoch 1/2
1719/1719 [==============================] - 15s 8ms/step - loss: 1.2926 - accuracy: 0.8764 - val_loss: 0.8167 - val_accuracy: 0.8898
Epoch 2/2
1719/1719 [==============================] - 13s 8ms/step - loss: 0.8209 - accuracy: 0.8851 - val_loss: 0.7753 - val_accuracy: 0.9026
